In [165]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

import inspect 

from fpdf import FPDF
import tempfile
import unidecode

In [148]:
def gerar_pdf(texto):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)

    for linha in texto.split("\n"):
        pdf.multi_cell(0, 10, linha)

    # Cria o arquivo temporário com extensão .pdf
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".pdf")
    caminho = tmp.name
    tmp.close()

    # Salva o conteúdo do PDF nesse caminho
    pdf.output(caminho)

    return caminho


In [ ]:
parametros_usados = {} 


def normalizar(texto):
    return unidecode.unidecode(str(texto)).strip().lower()

def consultar_notas(params: dict):
    caminho_csv = "Notas_alunos.csv"
    df = pd.read_csv(caminho_csv)

    turma = params.get("turma")
    ano_letivo = params.get("ano_letivo")
    matriculas = params.get("matriculas")
    alunos = params.get("alunos") or params.get("aluno")
    disciplina = params.get("disciplina")
    bimestre = params.get("bimestre")
    nota_maxima = params.get("nota_maxima", 7.0)
    tipo_consulta = params.get("tipo")  # "maior" ou "menor"

    df_filtrado = df.copy()

    # Normaliza colunas relevantes para garantir comparações robustas
    df_filtrado['aluno_normalizado'] = df_filtrado['aluno'].apply(normalizar)
    df_filtrado['disciplina_normalizada'] = df_filtrado['disciplina'].apply(normalizar)
    df_filtrado['turma_normalizada'] = df_filtrado['turma'].apply(normalizar)

    if turma:
        parametros_usados["turma"] = turma
        turma_norm = normalizar(turma)
        df_filtrado = df_filtrado[df_filtrado['turma_normalizada'] == turma_norm]

    if ano_letivo:
        parametros_usados["ano_letivo"] = ano_letivo
        df_filtrado = df_filtrado[df_filtrado['ano_letivo'] == int(ano_letivo)]

    if matriculas:
        parametros_usados["matriculas"] = matriculas
        df_filtrado = df_filtrado[df_filtrado['matricula'].isin(matriculas)]

    if alunos:
        if isinstance(alunos, str):
            alunos = [alunos]
        alunos_normalizados = [normalizar(a) for a in alunos]
        parametros_usados["alunos"] = alunos
        df_filtrado = df_filtrado[df_filtrado['aluno_normalizado'].apply(lambda x: any(a in x for a in alunos_normalizados))]

    if disciplina:
        disciplina_norm = normalizar(disciplina)
        parametros_usados["disciplina"] = disciplina
        df_filtrado = df_filtrado[df_filtrado['disciplina_normalizada'] == disciplina_norm]

    if df_filtrado.empty:
        return f"Nenhum aluno encontrado com os filtros aplicados: {parametros_usados}"

    # Consulta por maior/menor nota entre os bimestres
    if tipo_consulta in ["maior", "menor"]:
        notas_colunas = [col for col in df_filtrado.columns if col.startswith("nota_")]
        resultados = []

        for _, row in df_filtrado.iterrows():
            notas_validas = [(col, row[col]) for col in notas_colunas if pd.notna(row[col])]
            if not notas_validas:
                continue

            col_alvo, valor_alvo = max(notas_validas, key=lambda x: x[1]) if tipo_consulta == "maior" else min(notas_validas, key=lambda x: x[1])
            bimestre_desc = col_alvo.replace("nota_", "").replace("bim", "")

            resultados.append(
                f"{row['aluno']} teve a {tipo_consulta} nota {valor_alvo} em {row['disciplina']} no {bimestre_desc}"
            )

            parametros_usados["nota"] = valor_alvo
            parametros_usados["bimestre"] = bimestre_desc

        return "\n".join(resultados) if resultados else "Nenhuma nota encontrada."

    # Consulta padrão por bimestre
    if bimestre:
        coluna_nota = f'nota_{bimestre}bim'
        if coluna_nota not in df_filtrado.columns:
            return "Bimestre inválido."
        parametros_usados["bimestre"] = bimestre
        df_filtrado = df_filtrado[df_filtrado[coluna_nota] < float(nota_maxima)]

    respostas = []
    for _, row in df_filtrado.iterrows():
        nota = row.get(f'nota_{bimestre}bim') if bimestre else "N/A"
        respostas.append(
            f"{row['aluno']} (Matrícula: {row['matricula']}) - Nota: {nota} no {bimestre}º bimestre - Disciplina: {row['disciplina']} - Turma: {row['turma']}"
        )

        parametros_usados["nota"] = str(nota)

    return "\n".join(respostas)

In [124]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'ano': row['ano'],
        'bimestre': f"{row['bimestre']}° bimestre",
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents



[Document(id_='cdff94b1-be5e-45fe-9e9b-b2f48e2a8b34', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\npolinomiais do 1º\ngrau (função afim,\nfunção linear,\nfunção constante,\nfunção identidade)\nAplicação de Atividades via \nrecursos digitais\nRealização de Atividades via recursos \ndigitais', path=None, url=None, mimetype=None), image_resou

In [125]:
print(documents[0].get_metadata_str())

data: 2 ABR
ano: 2024
bimestre: 2° bimestre
materia: matematica


### Documentos em aprtes menores

In [126]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 2357.64it/s]


In [127]:
nodes

[TextNode(id_='c9a5b364-df13-4f00-b6a0-7a2683da2489', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cdff94b1-be5e-45fe-9e9b-b2f48e2a8b34', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, hash='6544dcb058080a680e9603c313c63688249e87f463da48cddbbc7e1899820825')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\

### Gerando Embeddings

In [128]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

In [129]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

'D:\\Documentos\\tcc - IA\\projeto\\all-MiniLM-L6-v2'

In [130]:
chroma_client = chromadb.Client()

In [131]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [132]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [133]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [134]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [135]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [136]:
load_dotenv(find_dotenv())

True

In [137]:
GROQ_API = os.environ.get("GROQ_API")

In [138]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [139]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=6)

#### Testando o modelo

In [140]:
from rapidfuzz import fuzz

In [206]:
def gerar_plano_estudo(parametros_usados):
    nota = float(parametros_usados.get('nota', 0))
    bimestre = parametros_usados.get('bimestre')
    materia = parametros_usados.get('materia')
    aluno = parametros_usados.get('alunos', 'aluno')

    if isinstance(aluno, list):
        aluno = aluno[0]

    if not bimestre or not materia:
        return "Informações insuficientes para gerar o plano de estudo (bimestre ou matéria ausente)."

    if nota >= 8:
        return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    # Filtra os documentos do bimestre e matéria corretos
    docs_filtrados = [
        doc for doc in documents
        if doc.metadata['bimestre'] == f"{bimestre}° bimestre"
        and fuzz.ratio(doc.metadata.get('materia', '').lower(), materia.lower()) >= 80
    ]

    if not docs_filtrados:
        return "Nenhum conteúdo encontrado para esse bimestre/matéria."

    # Cria novo index e query_engine só com os docs filtrados
    nodes_filtrados = node_parser.get_nodes_from_documents(docs_filtrados)
    index_filtrado = VectorStoreIndex(nodes_filtrados, embed_model=embed_model)
    query_engine_filtrado = index_filtrado.as_query_engine(llm=llm, similarity_top_k=6)

    prompt = f"""
    O aluno {aluno} obteve nota {nota} no {bimestre}º bimestre na matéria de {materia}.

    Você é um tutor inteligente. Gere um plano de estudo **EXCLUSIVAMENTE** com base no conteúdo de {materia.upper()} do **{bimestre}º bimestre**.

    Ignore completamente conteúdos de outros bimestres, mesmo que relacionados.
    Use **apenas** o que foi ensinado no {bimestre}º bimestre da matéria {materia.upper()}.

    Com base nisso, crie:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestões de atividades práticas

    Tudo em português e focado no nível do aluno com nota {nota} e ignore a data do texto, focando apenas no bimestre.
    """

    resp = query_engine_filtrado.query(prompt)
    return resp


In [64]:
gerar_plano_estudo(aluno="pedro", nota=6.5, bimestre=2, materia="Matematica", query_engine=query_engine)

TypeError: gerar_plano_estudo() got an unexpected keyword argument 'aluno'

In [207]:
import gradio as gr

def interface_plano_com_feedback(aluno, nota, bimestre, materia):
    try:
        plano = gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine)
        return plano
    except Exception as e:
        return f"Erro: {str(e)}"

with gr.Blocks() as demo:
    aluno = gr.Textbox(label="Nome do aluno")
    nota = gr.Number(label="Nota do aluno")
    bimestre = gr.Number(label="Bimestre")
    materia = gr.Textbox(label="Matéria")

    btn_gerar = gr.Button("Gerar Plano de Estudo")
    saida_plano = gr.Textbox(label="Plano de Estudo", lines=10)

    btn_gerar.click(interface_plano_com_feedback, inputs=[aluno, nota, bimestre, materia], outputs=saida_plano)

demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7887
* Running on public URL: https://ffa12b012b5a4a4d09.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [208]:
nota_tool = FunctionTool.from_defaults(
    fn=consultar_notas,
    name="ConsultarNotas",
    description="""
    Consulta notas de alunos a partir de um dicionário de parâmetros.

    Parâmetros possíveis:
    - aluno (str): nome do aluno
    - alunos (list[str]): lista de nomes
    - disciplina (str): nome da disciplina
    - bimestre (str ou int): bimestre (1 a 4)
    - tipo (str): 'maior' ou 'menor' para encontrar a maior ou menor nota do aluno
    - turma (str): nome da turma
    - ano_letivo (int): ano letivo
    - nota_maxima (float): filtro de nota máxima para alertas
    - nota (float): filtro de nota 

    Exemplos de uso:
    - "Qual a menor nota do aluno João?" → {'aluno': 'João', 'tipo': 'menor'}
    - "Nota de Maria no 2º bimestre em matemática" → {'aluno': 'Maria', 'bimestre': '2', 'disciplina': 'matemática'}
    """
)


In [209]:
retriever = index.as_retriever(similarity_top_k=6)
retriever_tool = QueryEngineTool.from_defaults(
    #query_engine=retriever,
    query_engine=index.as_query_engine(llm=llm, similarity_top_k=6),
    name="Planos de aula",
    description="Ferramenta que busca planos de aula, conteúdos pedagógicos e atividades relacionados à disciplina, bimestre, turma ou aluno. Use esta ferramenta para qualquer consulta sobre plano de ensino, conteúdo, planejamento, atividades ou reforço escolar.",
)

In [144]:
retriever_tool.query_engine.query("quero um plano de aula para um aluno que tem nota baixa em matematica do 1 bimestre fale em portugues")

Response(response='Considerando a necessidade de reforço em Matemática do 1° bimestre, proponho um plano de aula personalizado para o aluno.\n\n**Aula de Reforço em Matemática**\n\n**Objetivos:**\n\n* Revisar e consolidar conceitos fundamentais de Sistemas e unidades de medida, leitura e conversão de unidades de grandezas diversas.\n* Desenvolver habilidades de interpretação de dados e informações, envolvendo a variação entre grandezas.\n* Fortalecer a capacidade de resolver situações-problemas de temática trabalhada.\n\n**Metodologia:**\n\n1. Revisão dos conceitos fundamentais de Sistemas e unidades de medida, utilizando recursos visuais e materiais didáticos.\n2. Análise de gráficos e funções, destacando a variação entre grandezas e sua aplicação no cotidiano.\n3. Resolução de exercícios e situações-problemas, com foco em problemas de conversão de unidades e interpretação de dados.\n4. Atividades individuais e em grupo, para promover a interação e o debate sobre os conceitos trabalha

In [210]:


chat_engine = ReActAgent.from_tools(
    tools=[nota_tool],
    llm=llm,
    verbose=True
)
#index.as_query_engine(llm=llm,)


d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\llama_index\core\agent\react\base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
d:\Documentos\tcc - IA\projeto\venv\lib\site-packages\deprecated\classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


In [211]:

def responder(mensagem, chat_history):
    if "plano de aula" in mensagem:
        resposta = gerar_plano_estudo(parametros_usados)
        caminho = gerar_pdf(resposta)
        resposta = "Gerando pdf...\nsalvo em: " + caminho 
    else:
        resposta = chat_engine.chat(mensagem).response

    if chat_history is None:
        chat_history = []

    chat_history.append(("Você", mensagem))
    chat_history.append(("Assistente", resposta))
    
    return chat_history, chat_history

In [212]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(placeholder="Digite sua pergunta ou comando...")
    estado = gr.State([])


    entrada.submit(responder, inputs=[entrada, estado], outputs=[chatbot, estado])
    entrada.submit(lambda: "", None, entrada)

demo.launch()

C:\Users\andre\AppData\Local\Temp\ipykernel_24080\2862595693.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


> Running step f47f64d2-c2bf-42ac-8017-04d3251c51aa. Step input: qual a menor nota do KAMILLY VITORIA DOS REIS SOUZA em matematica?


Thought: The current language of the user is: Portuguese. I need to use a tool to help me answer the question.
Action: ConsultarNotas
Action Input: {'params': AttributedDict([('aluno', 'KAMILLY VITORIA DOS REIS SOUZA'), ('disciplina', 'matematica'), ('tipo', 'menor')])}
Observation: KAMILLY VITORIA DOS REIS SOUZA teve a menor nota 6.0 em MATEMÁTICA no 4º bimestre
> Running step 0090aef2-cb8a-4a3b-9d5f-b5475c2d9ec9. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: A menor nota do KAMILLY VITORIA DOS REIS SOUZA em matemática é 6.0 no 4º bimestre.


In [214]:
parametros_usados


{'alunos': ['KAMILLY VITORIA DOS REIS SOUZA'],
 'disciplina': 'matematica',
 'nota': 6.0,
 'bimestre': '4º bimestre'}